# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn import linear_model

In [2]:
pokemon = pd.read_csv("pokemon.csv")

In [3]:
display(pokemon.head(10))

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False


In [4]:
pokemon.isna().sum()

#               0
Name            0
Type 1          0
Type 2        386
Total           0
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64

In [5]:

def cleanup_pokemon_names(pokemon_names):
    cleaned_names = []
    for name in pokemon_names:
        if "Mega" in name:
            cleaned_name = name.replace(name.split("Mega", 1)[0], "Mega", 1).replace("MegaMega", "Mega")
            cleaned_names.append(cleaned_name.strip())
        else:
            cleaned_names.append(name)
    return cleaned_names

# Example list of Pokemon names
pokemon_names = pokemon['Name']

cleaned_names = cleanup_pokemon_names(pokemon_names)
for name in cleaned_names:
        (name)

pokemon['Name'] = cleanup_pokemon_names(pokemon['Name'])

display(pokemon.head(20))

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,Mega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
7,6,Mega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
8,6,Mega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False


In [6]:
pokemon = pokemon.drop(['Generation', 'Legendary'],axis=1)
pokemon

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80
3,3,Mega Venusaur,Grass,Poison,625,80,100,123,122,120,80
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50
796,719,Mega Diancie,Rock,Fairy,700,50,160,110,160,110,110
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80


In [7]:
pokemon.describe().T

,count,mean,std,min,25%,50%,75%,max
#,800.0,362.81375,208.343798,1.0,184.75,364.5,539.25,721.0
Total,800.0,435.10250,119.963040,180.0,330.00,450.0,515.00,780.0
HP,800.0,69.25875,25.534669,1.0,50.00,65.0,80.00,255.0
Attack,800.0,79.00125,32.457366,5.0,55.00,75.0,100.00,190.0
Defense,800.0,73.84250,31.183501,5.0,50.00,70.0,90.00,230.0
Sp. Atk,800.0,72.82000,32.722294,10.0,49.75,65.0,95.00,194.0
Sp. Def,800.0,71.90250,27.828916,20.0,50.00,70.0,90.00,230.0
Speed,800.0,68.27750,29.060474,5.0,45.00,65.0,90.00,180.0


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [8]:
pokemon.shape

(800, 11)

In [10]:
X_num = pokemon.select_dtypes(include = np.number)
X_cat = pokemon.select_dtypes(include = object)

In [11]:
X_num 

,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,318,45,49,49,65,65,45
1,2,405,60,62,63,80,80,60
2,3,525,80,82,83,100,100,80
3,3,625,80,100,123,122,120,80
4,4,309,39,52,43,60,50,65
...,...,...,...,...,...,...,...,...
795,719,600,50,100,150,100,150,50
796,719,700,50,160,110,160,110,110
797,720,600,80,110,60,150,130,70
798,720,680,80,160,60,170,130,80


In [12]:
from sklearn.preprocessing import MinMaxScaler
MinMaxtransformer = MinMaxScaler().fit(X_num)
X_normalized = MinMaxtransformer.transform(X_num)
print(type(X_normalized))
X_normalized = pd.DataFrame(X_normalized,columns=X_num.columns)
display(X_normalized.head())
print(type(X_normalized))

<class 'numpy.ndarray'>


,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,0.000000,0.230000,0.173228,0.237838,0.195556,0.298913,0.214286,0.228571
1,0.001389,0.375000,0.232283,0.308108,0.257778,0.380435,0.285714,0.314286
2,0.002778,0.575000,0.311024,0.416216,0.346667,0.489130,0.380952,0.428571
3,0.002778,0.741667,0.311024,0.513514,0.524444,0.608696,0.476190,0.428571
4,0.004167,0.215000,0.149606,0.254054,0.168889,0.271739,0.142857,0.342857


<class 'pandas.core.frame.DataFrame'>


In [13]:
X_cat = pokemon.select_dtypes(include = object)

In [14]:
X_cat.head()

,Name,Type 1,Type 2
0,Bulbasaur,Grass,Poison
1,Ivysaur,Grass,Poison
2,Venusaur,Grass,Poison
3,Mega Venusaur,Grass,Poison
4,Charmander,Fire,NaN


In [15]:
X_cat_Type1 = X_cat[['Type 1']]

In [16]:
X_cat_Type2 = X_cat[['Type 2']]

## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [17]:
from sklearn.preprocessing import OneHotEncoder
X_cat_combined = pd.concat([X_cat_Type1, X_cat_Type2], axis=1)
encoder = OneHotEncoder(drop='first').fit(X_cat_combined)
print(encoder.categories_)
encoded = encoder.transform(X_cat_combined).toarray()
print(encoded)


[array(['Bug', 'Dark', 'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire',
       'Flying', 'Ghost', 'Grass', 'Ground', 'Ice', 'Normal', 'Poison',
       'Psychic', 'Rock', 'Steel', 'Water'], dtype=object), array(['Bug', 'Dark', 'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire',
       'Flying', 'Ghost', 'Grass', 'Ground', 'Ice', 'Normal', 'Poison',
       'Psychic', 'Rock', 'Steel', 'Water', nan], dtype=object)]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


In [18]:
encoder = OneHotEncoder(drop='first').fit(X_cat_combined)

cols = encoder.get_feature_names_out(input_features=X_cat_combined.columns)

X_cat_combined_encode = pd.DataFrame(encoder.transform(X_cat_combined).toarray(),columns=cols)

X_cat_combined_encode.head()

,Type 1_Dark,Type 1_Dragon,Type 1_Electric,Type 1_Fairy,Type 1_Fighting,Type 1_Fire,Type 1_Flying,Type 1_Ghost,Type 1_Grass,Type 1_Ground,...,Type 2_Grass,Type 2_Ground,Type 2_Ice,Type 2_Normal,Type 2_Poison,Type 2_Psychic,Type 2_Rock,Type 2_Steel,Type 2_Water,Type 2_nan
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [19]:
X = pd.concat([X_normalized,X_cat_combined_encode], axis=1)

In [20]:
X

,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Type 1_Dark,Type 1_Dragon,...,Type 2_Grass,Type 2_Ground,Type 2_Ice,Type 2_Normal,Type 2_Poison,Type 2_Psychic,Type 2_Rock,Type 2_Steel,Type 2_Water,Type 2_nan
0,0.000000,0.230000,0.173228,0.237838,0.195556,0.298913,0.214286,0.228571,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.001389,0.375000,0.232283,0.308108,0.257778,0.380435,0.285714,0.314286,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.002778,0.575000,0.311024,0.416216,0.346667,0.489130,0.380952,0.428571,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.002778,0.741667,0.311024,0.513514,0.524444,0.608696,0.476190,0.428571,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.004167,0.215000,0.149606,0.254054,0.168889,0.271739,0.142857,0.342857,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0.997222,0.700000,0.192913,0.513514,0.644444,0.489130,0.619048,0.257143,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
796,0.997222,0.866667,0.192913,0.837838,0.466667,0.815217,0.428571,0.600000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
797,0.998611,0.700000,0.311024,0.567568,0.244444,0.760870,0.523810,0.371429,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
798,0.998611,0.833333,0.311024,0.837838,0.244444,0.869565,0.523810,0.428571,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [34]:
Total = X_cat_combined_encode
Total['Total'] = pokemon['Total']


In [35]:
Total

,Type 1_Dark,Type 1_Dragon,Type 1_Electric,Type 1_Fairy,Type 1_Fighting,Type 1_Fire,Type 1_Flying,Type 1_Ghost,Type 1_Grass,Type 1_Ground,...,Type 2_Ground,Type 2_Ice,Type 2_Normal,Type 2_Poison,Type 2_Psychic,Type 2_Rock,Type 2_Steel,Type 2_Water,Type 2_nan,Total
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,318
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,405
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,525
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,625
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600
796,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,700
797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600
798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,680


In [36]:
correlations_matrix =Total.corr()
correlations_matrix

,Type 1_Dark,Type 1_Dragon,Type 1_Electric,Type 1_Fairy,Type 1_Fighting,Type 1_Fire,Type 1_Flying,Type 1_Ghost,Type 1_Grass,Type 1_Ground,...,Type 2_Ground,Type 2_Ice,Type 2_Normal,Type 2_Poison,Type 2_Psychic,Type 2_Rock,Type 2_Steel,Type 2_Water,Type 2_nan,Total
Type 1_Dark,1.000000,-0.040984,-0.048438,-0.029584,-0.037524,-0.052938,-0.014233,-0.040984,-0.062173,-0.040984,...,-0.042946,0.071991,-0.014233,-0.042300,0.023490,-0.026796,0.045446,-0.026796,-0.064256,0.017818
Type 1_Dragon,-0.040984,1.000000,-0.049245,-0.030077,-0.038149,-0.053820,-0.014470,-0.041667,-0.063209,-0.041667,...,0.112272,0.118700,-0.014470,-0.043005,0.085964,-0.027243,-0.034325,-0.027243,-0.056679,0.196532
Type 1_Electric,-0.048438,-0.049245,1.000000,-0.035547,-0.045088,-0.063609,-0.017102,-0.049245,-0.074706,-0.049245,...,-0.051602,0.009617,0.138368,-0.050827,-0.050041,-0.032197,0.060014,0.009617,0.063312,0.016715
Type 1_Fairy,-0.029584,-0.030077,-0.035547,1.000000,-0.027538,-0.038850,-0.010445,-0.030077,-0.045628,-0.030077,...,-0.031517,-0.019665,-0.010445,-0.031043,-0.030563,-0.019665,-0.024778,-0.019665,0.117907,-0.026948
Type 1_Fighting,-0.037524,-0.038149,-0.045088,-0.027538,1.000000,-0.049277,-0.013248,-0.038149,-0.057873,-0.038149,...,-0.039976,-0.024943,-0.013248,-0.039375,0.065654,-0.024943,0.053226,-0.024943,0.096586,-0.029086
Type 1_Fire,-0.052938,-0.053820,-0.063609,-0.038850,-0.049277,1.000000,-0.018691,-0.053820,-0.081647,-0.053820,...,0.017973,-0.035189,0.125084,-0.055549,-0.003697,0.003480,-0.013332,0.003480,0.029528,0.050527
Type 1_Flying,-0.014233,-0.014470,-0.017102,-0.010445,-0.013248,-0.018691,1.000000,-0.014470,-0.021951,-0.014470,...,-0.015163,-0.009461,-0.005025,-0.014935,-0.014704,-0.009461,-0.011921,-0.009461,0.002483,0.029504
Type 1_Ghost,-0.040984,-0.041667,-0.049245,-0.030077,-0.038149,-0.053820,-0.014470,1.000000,-0.063209,-0.041667,...,-0.043661,-0.027243,-0.014470,0.083480,-0.042340,-0.027243,-0.034325,-0.027243,-0.069445,0.007594
Type 1_Grass,-0.062173,-0.063209,-0.074706,-0.045628,-0.057873,-0.081647,-0.021951,-0.063209,1.000000,-0.063209,...,-0.044608,0.059883,-0.021951,0.263700,-0.019742,-0.041328,0.002029,-0.041328,-0.006861,-0.036057
Type 1_Ground,-0.040984,-0.041667,-0.049245,-0.030077,-0.038149,-0.053820,-0.014470,-0.041667,-0.063209,1.000000,...,-0.043661,-0.027243,-0.014470,-0.043005,0.021812,0.118700,0.004681,-0.027243,-0.031148,0.004082


In [47]:
correlations_matrix = Total.corr()['Total']
sorted_correlations = correlations_matrix.sort_values(ascending=False)

Highest_type = sorted_correlations.index[1] 

Highest_type


'Type 1_Dragon'

In [58]:
correlations_matrix['Total']


Type 1_Dark        0.017818
Type 1_Dragon      0.196532
Type 1_Electric    0.016715
Type 1_Fairy      -0.026948
Type 1_Fighting   -0.029086
Type 1_Fire        0.050527
Type 1_Flying      0.029504
Type 1_Ghost       0.007594
Type 1_Grass      -0.036057
Type 1_Ground      0.004082
Type 1_Ice        -0.002412
Type 1_Normal     -0.104150
Type 1_Poison     -0.057123
Type 1_Psychic     0.094364
Type 1_Rock        0.037524
Type 1_Steel       0.082000
Type 1_Water      -0.015640
Type 2_Dark        0.065844
Type 2_Dragon      0.115240
Type 2_Electric    0.014669
Type 2_Fairy      -0.024606
Type 2_Fighting    0.138726
Type 2_Fire        0.073234
Type 2_Flying      0.054048
Type 2_Ghost      -0.004885
Type 2_Grass      -0.039224
Type 2_Ground      0.016486
Type 2_Ice         0.100870
Type 2_Normal     -0.013956
Type 2_Poison     -0.067837
Type 2_Psychic     0.076054
Type 2_Rock       -0.000512
Type 2_Steel       0.070307
Type 2_Water      -0.018800
Type 2_nan        -0.185945
Total              1

# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [62]:
Total.describe().T

,count,mean,std,min,25%,50%,75%,max
Type 1_Dark,800.0,0.03875,0.193119,0.0,0.0,0.0,0.0,1.0
Type 1_Dragon,800.0,0.04000,0.196082,0.0,0.0,0.0,0.0,1.0
Type 1_Electric,800.0,0.05500,0.228123,0.0,0.0,0.0,0.0,1.0
Type 1_Fairy,800.0,0.02125,0.144307,0.0,0.0,0.0,0.0,1.0
Type 1_Fighting,800.0,0.03375,0.180698,0.0,0.0,0.0,0.0,1.0
Type 1_Fire,800.0,0.06500,0.246680,0.0,0.0,0.0,0.0,1.0
Type 1_Flying,800.0,0.00500,0.070578,0.0,0.0,0.0,0.0,1.0
Type 1_Ghost,800.0,0.04000,0.196082,0.0,0.0,0.0,0.0,1.0
Type 1_Grass,800.0,0.08750,0.282743,0.0,0.0,0.0,0.0,1.0
Type 1_Ground,800.0,0.04000,0.196082,0.0,0.0,0.0,0.0,1.0


In [99]:
pokemon_grouped_bytypes.sort_values(by='Total', ascending=False)

,,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
Type 1,Type 2,,,,,,,,
Ground,Fire,383.0,770.000000,100.0,180.0,160.000000,150.0,90.000000,90.0
Dragon,Ice,646.0,686.666667,125.0,140.0,93.333333,140.0,93.333333,95.0
Steel,Dragon,483.0,680.000000,100.0,120.0,120.000000,150.0,100.000000,90.0
Ghost,Dragon,487.0,680.000000,150.0,110.0,110.000000,110.0,110.000000,90.0
Psychic,Dark,720.0,680.000000,80.0,160.0,60.000000,170.0,130.000000,80.0
...,...,...,...,...,...,...,...,...,...
Poison,Bug,451.0,330.000000,40.0,50.0,90.000000,30.0,55.000000,65.0
Normal,Fairy,216.4,330.000000,99.6,45.0,49.200000,54.0,52.200000,30.0
Poison,Water,690.0,320.000000,50.0,60.0,60.000000,60.0,60.000000,30.0


In [ ]:
The best Pokemom to choose is Ground Fire with a total of 770K. Just to remind that Total = Sum(HP	Attack	Defense	Sp. Atk	Sp. Def	Speed)